In [1]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_7d5d0d16a6454b0987a63e487c91c25b.csv"
df=pd.read_csv(url)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
df.shape

(768, 9)

In [3]:
df.isna().any()

Pregnancies                 False
Glucose                     False
BloodPressure               False
SkinThickness               False
Insulin                     False
BMI                         False
DiabetesPedigreeFunction    False
Age                         False
Outcome                     False
dtype: bool

In [4]:
df.groupby("Outcome").count().T

Outcome,0,1
Pregnancies,500,268
Glucose,500,268
BloodPressure,500,268
SkinThickness,500,268
Insulin,500,268
BMI,500,268
DiabetesPedigreeFunction,500,268
Age,500,268


In [5]:
X=df.drop(["Outcome"],axis=1)
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [6]:
y=df[["Outcome"]]
y.head()

,Outcome
0,1
1,0
2,1
3,0
4,1


In [7]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496


In [8]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

# feature extraction. We are going to output the selection scores for all features and select the features with the highest scores.
test = SelectKBest(score_func=mutual_info_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
1,0.107442
5,0.097107
7,0.059220
4,0.039363
0,0.025447
6,0.016045
3,0.009182
2,0.000000


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
print(X_train.shape)
print(X_test.shape)

(652, 8)
(116, 8)


In [10]:
df_t=pd.concat([X_train,y_train],axis=1)

In [11]:
df_t0=df_t[df_t["Outcome"]==0]
df_t1=df_t[df_t["Outcome"]==1]
df_t1=pd.concat([df_t1]*2,axis=0)

In [12]:
df_t=pd.concat([df_t0,df_t1],axis=0)
df_t.shape

(880, 9)

In [13]:
X_train=df_t.drop(["Outcome"],axis=1)
X_train.shape

(880, 8)

In [14]:
y_train=df_t[["Outcome"]]
y_train.shape

(880, 1)

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
gradient=GradientBoostingClassifier(loss="deviance",learning_rate=0.01,n_estimators=100,subsample=1,criterion="mse",min_samples_split=3,min_samples_leaf=2,max_depth=2,max_features="auto",validation_fraction=0.2,random_state=0).fit(X_train,y_train)
gradient

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='mse', init=None, learning_rate=0.01,
              loss='deviance', max_depth=2, max_features='auto',
              max_leaf_nodes=None, min_impurity_decrease=0.0,
              min_impurity_split=None, min_samples_leaf=2,
              min_samples_split=3, min_weight_fraction_leaf=0.0,
              n_estimators=100, n_iter_no_change=None, presort='auto',
              random_state=0, subsample=1, tol=0.0001,
              validation_fraction=0.2, verbose=0, warm_start=False)

In [16]:
from sklearn.metrics import accuracy_score
y0_pred=gradient.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=gradient.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.7909090909090909
Accuracy for test data is:  0.7672413793103449


In [18]:
from sklearn.metrics import roc_auc_score
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.7903012247600132
AUC for test data is:  0.769078947368421


In [19]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[328  96]
 [ 88 368]]
Confusion matrix for test data is: 
 [[58 18]
 [ 9 31]]


In [20]:
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(n_estimators=300,criterion="gini",max_depth=4,min_samples_split=3,min_samples_leaf=3,max_features="auto",bootstrap=True,n_jobs=-1,warm_start=True,class_weight="balanced_subsample",random_state=0).fit(X_train,y_train)
forest

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:502: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '


RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=4, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=3,
            min_samples_split=3, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=-1, oob_score=False, random_state=0,
            verbose=0, warm_start=True)

In [21]:
y0_pred=forest.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=forest.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.8352272727272727
Accuracy for test data is:  0.75


In [22]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.8338091691492883
AUC for test data is:  0.7677631578947368


In [23]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[337  87]
 [ 58 398]]
Confusion matrix for test data is: 
 [[54 22]
 [ 7 33]]


In [24]:
from sklearn.naive_bayes import GaussianNB
NB=GaussianNB().fit(X_train,y_train)
NB

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None, var_smoothing=1e-09)

In [25]:
y0_pred=NB.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=NB.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.7227272727272728
Accuracy for test data is:  0.7155172413793104


In [26]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.7240152267461105
AUC for test data is:  0.7236842105263158


In [27]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[322 102]
 [142 314]]
Confusion matrix for test data is: 
 [[53 23]
 [10 30]]


In [36]:
from sklearn.gaussian_process import GaussianProcessClassifier
Gaussian=GaussianProcessClassifier(n_restarts_optimizer=3,max_iter_predict=100,warm_start=True,multi_class="one_vs_one",n_jobs=-1,random_state=0).fit(X_train,y_train)
Gaussian

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianProcessClassifier(copy_X_train=True, kernel=None,
             max_iter_predict=100, multi_class='one_vs_one', n_jobs=-1,
             n_restarts_optimizer=3, optimizer='fmin_l_bfgs_b',
             random_state=0, warm_start=True)

In [37]:
y0_pred=Gaussian.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=Gaussian.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.9022727272727272
Accuracy for test data is:  0.6896551724137931


In [38]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.9012330354187356
AUC for test data is:  0.7098684210526316


In [39]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[370  54]
 [ 32 424]]
Confusion matrix for test data is: 
 [[49 27]
 [ 9 31]]


In [40]:
from sklearn.linear_model import LogisticRegression
logistic=LogisticRegression(penalty="l2",C=0.1,class_weight="balanced",solver="saga",max_iter=200,multi_class="multinomial",n_jobs=-1,random_state=0).fit(X_train,y_train)
logistic

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=0.1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=200,
          multi_class='multinomial', n_jobs=-1, penalty='l2',
          random_state=0, solver='saga', tol=0.0001, verbose=0,
          warm_start=False)

In [41]:
y0_pred=logistic.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=logistic.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.7568181818181818
Accuracy for test data is:  0.7068965517241379


In [42]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.757406487917908
AUC for test data is:  0.7171052631578948


In [43]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[328  96]
 [118 338]]
Confusion matrix for test data is: 
 [[52 24]
 [10 30]]


In [44]:
from sklearn import svm
svm=svm.SVC(C=0.2,kernel="poly",degree=3,gamma="scale",cache_size=200,decision_function_shape="ovo",random_state=0).fit(X_train,y_train)
svm

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='scale', kernel='poly',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False)

In [45]:
y0_pred=svm.predict(X_train)
print("Accuracy for train data is: ",accuracy_score(y_train,y0_pred))
y1_pred=svm.predict(X_test)
print("Accuracy for test data is: ", accuracy_score(y_test,y1_pred))

Accuracy for train data is:  0.7397727272727272
Accuracy for test data is:  0.7758620689655172


In [46]:
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.7458416087388282
AUC for test data is:  0.7401315789473684


In [47]:
print("Confusion matrix for for train data is: \n",confusion_matrix(y_train,y0_pred))
print("Confusion matrix for test data is: \n", confusion_matrix(y_test,y1_pred))

Confusion matrix for for train data is: 
 [[387  37]
 [192 264]]
Confusion matrix for test data is: 
 [[65 11]
 [15 25]]


In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam

In [18]:
length=X_test.shape[1]
length

8

In [19]:
from keras.utils import to_categorical
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
num_classes=y_test.shape[1]
print(num_classes)

2


In [62]:
def classification_model():
    # create model
    model = Sequential()
    model.add(Dense(length, activation='relu', input_shape=(length,)))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.33))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.33))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.33))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    opt=Adam(lr=0.01)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [70]:
model=classification_model()
model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=10, verbose=2)
model.evaluate(X_test,y_test,verbose=0)

Train on 880 samples, validate on 116 samples
Epoch 1/10
 - 9s - loss: 0.6468 - acc: 0.6614 - val_loss: 0.6420 - val_acc: 0.7155
Epoch 2/10
 - 1s - loss: 0.5675 - acc: 0.7125 - val_loss: 0.5620 - val_acc: 0.6983
Epoch 3/10
 - 1s - loss: 0.5325 - acc: 0.7500 - val_loss: 0.5286 - val_acc: 0.7500
Epoch 4/10
 - 1s - loss: 0.5280 - acc: 0.7489 - val_loss: 0.6713 - val_acc: 0.6897
Epoch 5/10
 - 1s - loss: 0.5154 - acc: 0.7545 - val_loss: 0.5353 - val_acc: 0.7586
Epoch 6/10
 - 1s - loss: 0.4878 - acc: 0.7568 - val_loss: 0.5930 - val_acc: 0.7414
Epoch 7/10
 - 1s - loss: 0.5006 - acc: 0.7670 - val_loss: 0.6245 - val_acc: 0.7155
Epoch 8/10
 - 1s - loss: 0.4691 - acc: 0.7716 - val_loss: 0.5728 - val_acc: 0.7241
Epoch 9/10
 - 1s - loss: 0.4599 - acc: 0.7648 - val_loss: 0.7789 - val_acc: 0.6466
Epoch 10/10
 - 1s - loss: 0.5044 - acc: 0.7580 - val_loss: 0.5727 - val_acc: 0.7500


[0.5727339654133238, 0.75]

In [71]:
from sklearn.metrics import roc_auc_score
y0_pred=model.predict(X_train)
print("AUC for train data is: ",roc_auc_score(y_train,y0_pred))
y1_pred=model.predict(X_test)
print("AUC for test data is: ", roc_auc_score(y_test,y1_pred))

AUC for train data is:  0.8650074478649454
AUC for test data is:  0.8348684210526316


In [72]:
y1_pred=round(pd.DataFrame(y1_pred).rename(columns={0:"Pred_0",1:"Pred_1"}),1)
y1_pred.head()

,Pred_0,Pred_1
0,0.7,0.3
1,0.7,0.3
2,0.9,0.1
3,0.6,0.4
4,0.4,0.6


In [73]:
y1_pred["Pred_0"]=np.where((y1_pred.Pred_0 >=0.5),1,y1_pred.Pred_0)
y1_pred["Pred_0"]=np.where((y1_pred.Pred_0 < 0.5),0,y1_pred.Pred_0)
y1_pred["Pred_1"]=np.where((y1_pred.Pred_1 >=0.5),1,y1_pred.Pred_1)
y1_pred["Pred_1"]=np.where((y1_pred.Pred_1 <0.5),0,y1_pred.Pred_1)
y1_pred.head()

,Pred_0,Pred_1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [74]:
y1_pred["Pred_0"]=np.where((y1_pred.Pred_0 == y1_pred.Pred_1),0,y1_pred.Pred_0)
y1_pred.head()

,Pred_0,Pred_1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [75]:
y_test=pd.DataFrame(y_test).rename(columns={0:"Actual_0",1:"Actual_1"})
y_test.head()

,Actual_0,Actual_1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [76]:
len0=len(y_test[y_test["Actual_0"]==1])
print("The number of actual negative cases: ",len0)

The number of actual negative cases:  76


In [77]:
len1=len(y_test[y_test["Actual_1"]==1])
print("The number of actual positive cases: ",len1)

The number of actual positive cases:  40


In [78]:
len00=len(y1_pred[y1_pred["Pred_0"]==1])
print("The number of predicted negative cases: ",len00)

The number of predicted negative cases:  53


In [79]:
len11=len(y1_pred[y1_pred["Pred_1"]==1])
print("The number of predicted positive cases: ",len11)

The number of predicted positive cases:  63


In [81]:
print("Confusion matrix for test data: \n",np.array([[53,23],[0,40]]))

Confusion matrix for test data: 
 [[53 23]
 [ 0 40]]
